In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Machine Learning Fundamentals with BigQuery DataFrames

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/googleapis/python-bigquery-dataframes/blob/main/notebooks/getting_started/ml_fundamentals_bq_dataframes.ipynb">
      <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/Google_Colaboratory_SVG_Logo.svg/1600px-Google_Colaboratory_SVG_Logo.svg.png?20221103151432" alt="Colab logo" width="65"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/getting_started/ml_fundamentals_bq_dataframes.ipynb">
      <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Octicons-mark-github.svg/1200px-Octicons-mark-github.svg.png?20180806170715" alt="GitHub logo" width="35">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/getting_started/ml_fundamentals_bq_dataframes.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/googleapis/python-bigquery-dataframes/blob/main/notebooks/getting_started/ml_fundamentals_bq_dataframes.ipynb">
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW1gvOovVlbZAIZylUtf5Iu8-693qS1w5NJw&s" alt="BQ logo" width="35">
      Open in BQ Studio
    </a>
  </td>
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.10

## Overview

The `bigframes.ml` module implements Scikit-Learn's machine learning API in
BigQuery DataFrames. It exposes BigQuery's ML capabilities in a simple, popular
API that works seamlessly with the rest of the BigQuery DataFrames API.

Learn more about [BigQuery DataFrames](https://cloud.google.com/python/docs/reference/bigframes/latest).

### Objective

In this tutorial, you will walk through an end-to-end machine learning workflow using BigQuery DataFrames. You will load data, manipulate and prepare it for model training, build supervised and unsupervised models, and evaluate and save a model for future use; all using built-in BigQuery DataFrames functionality.

### Dataset

This tutorial uses the [```penguins``` table](https://console.cloud.google.com/bigquery?p=bigquery-public-data&d=ml_datasets&t=penguins) (a BigQuery public dataset), which contains data on a set of penguins including species, island of residence, weight, culmen length and depth, flipper length, and sex.

### Costs

This tutorial uses billable components of Google Cloud:

* BigQuery (storage and compute)
* BigQuery ML

Learn about [BigQuery storage pricing](https://cloud.google.com/bigquery/pricing#storage),
[BigQuery compute pricing](https://cloud.google.com/bigquery/pricing#analysis_pricing_models),
and [BigQuery ML pricing](https://cloud.google.com/bigquery/pricing#bqml),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Depending on your Jupyter environment, you might have to install packages.

**Vertex AI Workbench or Colab**

Do nothing, BigQuery DataFrames package is already installed.

**Local JupyterLab instance**

Uncomment and run the following cell:

In [2]:
# !pip install bigframes

## Before you begin

Complete the tasks in this section to set up your environment.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Click here](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com) to enable the BigQuery API.

4. If you are running this notebook locally, install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

If you don't know your project ID, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113).

In [3]:
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Set the region

You can also change the `REGION` variable used by BigQuery. Learn more about [BigQuery regions](https://cloud.google.com/bigquery/docs/locations#supported_locations).

In [4]:
REGION = "US"  # @param {type: "string"}

#### Set the dataset ID

As part of this notebook, you will save BigQuery ML models to your Google Cloud project, which requires a dataset. Create the dataset, if needed, and provide the ID here as the `DATASET` variable used by BigQuery. Learn how to create a [BigQuery dataset](https://cloud.google.com/bigquery/docs/datasets).

In [5]:
DATASET = ""  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you might have to manually authenticate. Follow the relevant instructions below.

**Vertex AI Workbench**

Do nothing, you are already authenticated.

**Local JupyterLab instance**

Uncomment and run the following cell:

In [6]:
# ! gcloud auth login

**Colab**

Uncomment and run the following cell:

In [7]:
# from google.colab import auth
# auth.authenticate_user()

### Import libraries

In [8]:
import bigframes.pandas as bpd


### Set BigQuery DataFrames options

In [9]:
# Note: The project option is not required in all environments.
# On BigQuery Studio, the project ID is automatically detected.
bpd.options.bigquery.project = PROJECT_ID

# Note: The location option is not required.
# It defaults to the location of the first table or query
# passed to read_gbq(). For APIs where a location can't be
# auto-detected, the location defaults to the "US" location.
bpd.options.bigquery.location = REGION

If you want to reset the location of the created DataFrame or Series objects, reset the session by executing `bpd.reset_session()`. After that, you can reuse `bpd.options.bigquery.location` to specify another location.

## Import data into BigQuery DataFrames

You can create a DataFrame by reading data from a BigQuery table.

In [10]:
df = bpd.read_gbq("bigquery-public-data.ml_datasets.penguins")
df = df.dropna()

# BigQuery DataFrames creates a default numbered index, which we can give a name
df.index.name = "penguin_id"

Take a look at a few rows of the DataFrame:

In [11]:
df.head()

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
penguin_id,,,,,,,
0,Gentoo penguin (Pygoscelis papua),Biscoe,50.5,15.9,225.0,5400.0,MALE
1,Gentoo penguin (Pygoscelis papua),Biscoe,45.1,14.5,215.0,5000.0,FEMALE
2,Adelie Penguin (Pygoscelis adeliae),Torgersen,41.4,18.5,202.0,3875.0,MALE
3,Adelie Penguin (Pygoscelis adeliae),Torgersen,38.6,17.0,188.0,2900.0,FEMALE
4,Gentoo penguin (Pygoscelis papua),Biscoe,46.5,14.8,217.0,5200.0,FEMALE


## Clean and prepare data

We're are going to start with supervised learning, where a Linear Regression model will learn to predict the body mass (output variable `y`) using input features such as flipper length, sex, species, and more (features `X`).

In [12]:
# Isolate input features and output variable into DataFrames
X = df[['island', 'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'sex', 'species']]
y = df[['body_mass_g']]

Part of preparing data for a machine learning task is splitting it into subsets for training and testing to ensure that the solution is not overfitting. By default, BQML will automatically manage splitting the data for you. However, BQML also supports manually splitting out your training data.

Performing a manual data split can be done with `bigframes.ml.model_selection.train_test_split` like so:

In [13]:
from bigframes.ml.model_selection import train_test_split

# This will split X and y into test and training sets, with 20% of the rows in the test set,
# and the rest in the training set
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.2)

# Show the shape of the data after the split
print(f"""X_train shape: {X_train.shape}
X_test shape: {X_test.shape}
y_train shape: {y_train.shape}
y_test shape: {y_test.shape}""")

X_train shape: (267, 6)
X_test shape: (67, 6)
y_train shape: (267, 1)
y_test shape: (67, 1)


If we look at the data, we can see that random rows were selected for
each side of the split:

In [14]:
X_test.head(5)

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,sex,species
penguin_id,,,,,,
188,Dream,51.5,18.7,187.0,MALE,Chinstrap penguin (Pygoscelis antarctica)
251,Biscoe,49.5,16.1,224.0,MALE,Gentoo penguin (Pygoscelis papua)
231,Biscoe,45.7,13.9,214.0,FEMALE,Gentoo penguin (Pygoscelis papua)
271,Biscoe,59.6,17.0,230.0,MALE,Gentoo penguin (Pygoscelis papua)
128,Biscoe,38.8,17.2,180.0,MALE,Adelie Penguin (Pygoscelis adeliae)


Note that the `y_test` data matches the same rows in `X_test`:

In [15]:
y_test.head(5)

,body_mass_g
penguin_id,
188,3250.0
251,5650.0
231,4400.0
271,6050.0
128,3800.0


## Estimators

Following scikit-learn, all learning components are "estimators"; objects that can learn from training data and then apply themselves to new data. Estimators share the following patterns:

- a constructor that takes a list of parameters
- a standard string representation that shows the class name and all non-default parameters, e.g. `LinearRegression(fit_intercept=False)`
- a `.fit(..)` method to fit the estimator to training data

There estimators can be further broken down into two main subtypes:
 1. Transformers
 2. Predictors

Let's walk through each of these with our example model.

### Transformers

Transformers are estimators that are used to prepare data for consumption by other estimators ('preprocessing'). In addition to `.fit(...)`, the transformer implements a `.transform(...)` method, which will apply a transformation based on what was computed during `.fit(..)`. With this pattern dynamic preprocessing steps can be applied to both training and test/production data consistently.

An example of a transformer is `bigframes.ml.preprocessing.StandardScaler`, which rescales a dataset to have a mean of zero and a standard deviation of one:

In [16]:
from bigframes.ml.preprocessing import StandardScaler

# StandardScaler will only work on numeric columns
numeric_columns = ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm"]

scaler = StandardScaler()
scaler.fit(X_train[numeric_columns])

# Now, standardscaler should transform the numbers to have mean of zero
# and standard deviation of one:
scaler.transform(X_train[numeric_columns])

,standard_scaled_culmen_length_mm,standard_scaled_culmen_depth_mm,standard_scaled_flipper_length_mm
penguin_id,,,
0,1.20778,-0.651531,1.772656
2,-0.455602,0.662855,0.100476
3,-0.967412,-0.095445,-0.917372
4,0.476623,-1.207617,1.191028
5,-1.625454,0.359535,-0.626559
7,-0.345929,-1.86481,0.682104
8,0.842202,-1.561491,1.409139
9,0.348671,0.865068,-0.263041
10,0.933596,1.218941,0.827511


We can then repeat this transformation on the test data:

In [17]:
scaler.transform(X_test[numeric_columns])

,standard_scaled_culmen_length_mm,standard_scaled_culmen_depth_mm,standard_scaled_flipper_length_mm
penguin_id,,,
1,0.220718,-1.359277,1.045621
15,-0.510439,0.157322,-0.771966
28,-1.058807,0.713408,-0.771966
32,1.463685,1.168388,0.39129
33,-0.254534,0.056215,-0.990076
34,-0.510439,0.460642,0.318587
37,1.354012,0.511195,-0.263041
41,-0.674949,-0.095445,-1.789814
47,-1.168481,0.662855,-0.117634


#### Composing transformers

To process data where different columns need different preprocessors, `bigframes.composition.ColumnTransformer` can be employed.

Let's create an aggregate transform that applies `StandardScalar` to the numeric columns and `OneHotEncoder` to the string columns.

In [18]:
from bigframes.ml.compose import ColumnTransformer
from bigframes.ml.preprocessing import OneHotEncoder

# Create an aggregate transform that applies StandardScaler to the numeric columns,
# and OneHotEncoder to the string columns
preproc = ColumnTransformer([
    ("scale", StandardScaler(), ["culmen_length_mm", "culmen_depth_mm", "flipper_length_mm"]),
    ("encode", OneHotEncoder(), ["species", "sex", "island"])])

# Now we can fit all columns of the training data
preproc.fit(X_train)

processed_X_train = preproc.transform(X_train)
processed_X_test = preproc.transform(X_test)

# View the processed training data
processed_X_train

,onehotencoded_island,standard_scaled_culmen_length_mm,standard_scaled_culmen_depth_mm,standard_scaled_flipper_length_mm,onehotencoded_sex,onehotencoded_species
penguin_id,,,,,,
0,"[{'index': 1, 'value': 1.0}]",1.20778,-0.651531,1.772656,"[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
2,"[{'index': 3, 'value': 1.0}]",-0.455602,0.662855,0.100476,"[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
3,"[{'index': 3, 'value': 1.0}]",-0.967412,-0.095445,-0.917372,"[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
4,"[{'index': 1, 'value': 1.0}]",0.476623,-1.207617,1.191028,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
5,"[{'index': 1, 'value': 1.0}]",-1.625454,0.359535,-0.626559,"[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
7,"[{'index': 1, 'value': 1.0}]",-0.345929,-1.86481,0.682104,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
8,"[{'index': 1, 'value': 1.0}]",0.842202,-1.561491,1.409139,"[{'index': 3, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
9,"[{'index': 3, 'value': 1.0}]",0.348671,0.865068,-0.263041,"[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
10,"[{'index': 2, 'value': 1.0}]",0.933596,1.218941,0.827511,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"


### Predictors

Predictors are estimators that learn and make predictions. In addition to `.fit(...)`, the predictor implements a `.predict(...)` method, which will use what was learned during `.fit(...)` to predict some output.

Predictors can be further broken down into two categories:
* Supervised predictors
* Unsupervised predictors

#### Supervised predictors

Supervised learning is when we train a model on input-output pairs, and then ask it to predict the output for new inputs. An example of such a predictor is `bigframes.ml.linear_models.LinearRegression`.

In [19]:
from bigframes.ml.linear_model import LinearRegression

linreg = LinearRegression()

# Learn from the training data how to predict output y
linreg.fit(processed_X_train, y_train)

# Predict y for the test data
predicted_y_test = linreg.predict(processed_X_test)

# View predictions
predicted_y_test

,predicted_body_mass_g,onehotencoded_island,standard_scaled_culmen_length_mm,standard_scaled_culmen_depth_mm,standard_scaled_flipper_length_mm,onehotencoded_sex,onehotencoded_species
penguin_id,,,,,,,
1,4772.376044,"[{'index': 1, 'value': 1.0}]",0.220718,-1.359277,1.045621,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
15,3883.373922,"[{'index': 2, 'value': 1.0}]",-0.510439,0.157322,-0.771966,"[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
28,3479.709088,"[{'index': 2, 'value': 1.0}]",-1.058807,0.713408,-0.771966,"[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
32,4223.853626,"[{'index': 2, 'value': 1.0}]",1.463685,1.168388,0.39129,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
33,3197.623474,"[{'index': 2, 'value': 1.0}]",-0.254534,0.056215,-0.990076,"[{'index': 2, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
34,4155.26742,"[{'index': 2, 'value': 1.0}]",-0.510439,0.460642,0.318587,"[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
37,3991.314095,"[{'index': 2, 'value': 1.0}]",1.354012,0.511195,-0.263041,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
41,3232.648242,"[{'index': 3, 'value': 1.0}]",-0.674949,-0.095445,-1.789814,"[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
47,4017.740788,"[{'index': 2, 'value': 1.0}]",-1.168481,0.662855,-0.117634,"[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"


#### Unsupervised predictors

In unsupervised learning, there are no known outputs in the training data, instead the model learns on input data alone and predicts something else. An example of an unsupervised predictor is `bigframes.ml.cluster.KMeans`, which learns how to fit input data to a target number of clusters.

In [20]:
from bigframes.ml.cluster import KMeans

# Specify KMeans with four clusters
kmeans = KMeans(n_clusters=4)

# Fit data
kmeans.fit(processed_X_train)

# View predictions
kmeans.predict(processed_X_test)

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,onehotencoded_island,standard_scaled_culmen_length_mm,standard_scaled_culmen_depth_mm,standard_scaled_flipper_length_mm,onehotencoded_sex,onehotencoded_species
penguin_id,,,,,,,,
1,3,"[{'CENTROID_ID': 3, 'DISTANCE': 0.857057881337...","[{'index': 1, 'value': 1.0}]",0.220718,-1.359277,1.045621,"[{'index': 2, 'value': 1.0}]","[{'index': 3, 'value': 1.0}]"
15,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.181613302004...","[{'index': 2, 'value': 1.0}]",-0.510439,0.157322,-0.771966,"[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
28,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.006856853050...","[{'index': 2, 'value': 1.0}]",-1.058807,0.713408,-0.771966,"[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
32,2,"[{'CENTROID_ID': 2, 'DISTANCE': 1.237504384283...","[{'index': 2, 'value': 1.0}]",1.463685,1.168388,0.39129,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
33,2,"[{'CENTROID_ID': 2, 'DISTANCE': 1.656439702919...","[{'index': 2, 'value': 1.0}]",-0.254534,0.056215,-0.990076,"[{'index': 2, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
34,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.343792119214...","[{'index': 2, 'value': 1.0}]",-0.510439,0.460642,0.318587,"[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
37,2,"[{'CENTROID_ID': 2, 'DISTANCE': 0.816670297369...","[{'index': 2, 'value': 1.0}]",1.354012,0.511195,-0.263041,"[{'index': 3, 'value': 1.0}]","[{'index': 2, 'value': 1.0}]"
41,1,"[{'CENTROID_ID': 1, 'DISTANCE': 1.317560921596...","[{'index': 3, 'value': 1.0}]",-0.674949,-0.095445,-1.789814,"[{'index': 2, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"
47,4,"[{'CENTROID_ID': 4, 'DISTANCE': 1.135112005343...","[{'index': 2, 'value': 1.0}]",-1.168481,0.662855,-0.117634,"[{'index': 3, 'value': 1.0}]","[{'index': 1, 'value': 1.0}]"


## Pipelines

Transfomers and predictors can be chained into a single estimator component using `bigframes.ml.pipeline.Pipeline`:

In [21]:
from bigframes.ml.pipeline import Pipeline

pipeline = Pipeline([
  ('preproc', preproc),
  ('linreg', linreg)
])

# Print our pipeline
pipeline

Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  ['culmen_length_mm',
                                                   'culmen_depth_mm',
                                                   'flipper_length_mm']),
                                                 ('encode', OneHotEncoder(),
                                                  ['species', 'sex',
                                                   'island'])])),
                ('linreg', LinearRegression())])

The pipeline simplifies the workflow by applying each of its component steps automatically:

In [22]:
pipeline.fit(X_train, y_train)

predicted_y_test = pipeline.predict(X_test)
predicted_y_test

,predicted_body_mass_g,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,sex,species
penguin_id,,,,,,,
1,4772.374547,Biscoe,45.1,14.5,215.0,FEMALE,Gentoo penguin (Pygoscelis papua)
15,3883.371052,Dream,41.1,17.5,190.0,MALE,Adelie Penguin (Pygoscelis adeliae)
28,3479.706166,Dream,38.1,18.6,190.0,FEMALE,Adelie Penguin (Pygoscelis adeliae)
32,4223.851137,Dream,51.9,19.5,206.0,MALE,Chinstrap penguin (Pygoscelis antarctica)
33,3197.620461,Dream,42.5,17.3,187.0,FEMALE,Chinstrap penguin (Pygoscelis antarctica)
34,4155.265191,Dream,41.1,18.1,205.0,MALE,Adelie Penguin (Pygoscelis adeliae)
37,3991.311319,Dream,51.3,18.2,197.0,MALE,Chinstrap penguin (Pygoscelis antarctica)
41,3232.644783,Torgersen,40.2,17.0,176.0,FEMALE,Adelie Penguin (Pygoscelis adeliae)
47,4017.738303,Dream,37.5,18.5,199.0,MALE,Adelie Penguin (Pygoscelis adeliae)


In the backend, a pipeline will actually be compiled into a single model with an embedded TRANSFORM step.

## Evaluating results

Some models include a convenient `.score(X, y)` method for evaulation with a preset accuracy metric:

In [23]:
# In the case of a pipeline, this will be equivalent to calling .score on the contained LinearRegression
pipeline.score(X_test, y_test)

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,225.883512,77765.989281,0.004457,179.548041,0.873166,0.873315


For a more general approach, the library `bigframes.ml.metrics` is provided:

In [24]:
from bigframes.ml.metrics import r2_score

r2_score(y_test, predicted_y_test["predicted_body_mass_g"])

0.8731660699616813

## Save to BigQuery

Estimators can be saved to BigQuery as BQML models, and loaded again in future.

Saving requires `bigquery.tables.create` permission, and loading requires `bigquery.models.getMetadata` permission.
These permissions can be at project level or the dataset level.

If you have those permissions, please go ahead and uncomment the code in the following cells and run.

In [25]:
linreg.to_gbq(f"{DATASET}.penguins_model", replace=True)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('ont_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
                                                  'island'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_length_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_depth_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'flipper_length_mm'),
                                              

In [26]:
bpd.read_gbq_model(f"{DATASET}.penguins_model")

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('ont_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
                                                  'island'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_length_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_depth_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'flipper_length_mm'),
                                              

We can also save the pipeline to BigQuery. BigQuery will save this as a single model, with the pre-processing steps embedded in the TRANSFORM property:

In [27]:
pipeline.to_gbq(f"{DATASET}.penguins_pipeline", replace=True)

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('ont_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
                                                  'island'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_length_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_depth_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'flipper_length_mm'),
                                              

In [28]:
bpd.read_gbq_model(f"{DATASET}.penguins_pipeline")

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('ont_hot_encoder',
                                                  OneHotEncoder(max_categories=1000001,
                                                                min_frequency=0),
                                                  'island'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_length_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'culmen_depth_mm'),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  'flipper_length_mm'),
                                              

## Summary and next steps

You've completed an end-to-end machine learning workflow using the built-in capabilities of BigQuery DataFrames.

Learn more about BigQuery DataFrames in the [documentation](https://cloud.google.com/python/docs/reference/bigframes/latest) and find more sample notebooks in the [GitHub repo](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks).

### Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can uncomment the remaining cells and run them to delete the individual resources you created in this tutorial:

In [29]:
# # Delete the BQML models
# MODEL_NAME = f"{PROJECT_ID}:{DATASET}.penguins_model"
# ! bq rm -f --model {MODEL_NAME}
# PIPELINE_NAME = f"{PROJECT_ID}:{DATASET}.penguins_pipeline"
# ! bq rm -f --model {PIPELINE_NAME}